In [1]:
!pip install PyPDF2


     ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
     ----- --------------------------------- 30.7/232.6 kB 1.3 MB/s eta 0:00:01
     --------- --------------------------- 61.4/232.6 kB 656.4 kB/s eta 0:00:01
     --------------- -------------------- 102.4/232.6 kB 837.8 kB/s eta 0:00:01
     --------------------------------- ---- 204.8/232.6 kB 1.1 MB/s eta 0:00:01
     -------------------------------------- 232.6/232.6 kB 1.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
!git add .
!git commit -m "Update "
!git push

[main bf79987] Update clean word df
 Committer: Lê Thu Thủy PTCL <thuylt16@f88.vn>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly. Run the
following command and follow the instructions in your editor to edit
your configuration file:

    git config --global --edit

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 1 file changed, 1 insertion(+), 1 deletion(-)


remote: 
remote: GitHub found 5 vulnerabilities on Thuylt185411/web_lor_tracy's default branch (1 high, 3 moderate, 1 low). To find out more, visit:        
remote:      https://github.com/Thuylt185411/web_lor_tracy/security/dependabot        
remote: 
To https://github.com/Thuylt185411/web_lor_tracy.git
   46bf665..bf79987  main -> main


In [4]:
!git push -f origin main

remote: warning: See https://gh.io/lfs for more information.        
remote: warning: File 7dcaf9c4d3374b4c1503b4be66ba1f38dcb78487 is 86.40 MB; this is larger than GitHub's recommended maximum file size of 50.00 MB        
remote: warning: File 031e95d5d601e01d1d11cbf4b5e59cd5cc02fecc is 86.42 MB; this is larger than GitHub's recommended maximum file size of 50.00 MB        
remote: warning: GH001: Large files detected. You may want to try Git Large File Storage - https://git-lfs.github.com.        
remote: 
remote: GitHub found 2 vulnerabilities on Thuylt185411/web_lor_tracy's default branch (1 moderate, 1 low). To find out more, visit:        
remote:      https://github.com/Thuylt185411/web_lor_tracy/security/dependabot        
remote: 
To https://github.com/Thuylt185411/web_lor_tracy.git
 + 8d86686...2f190ef main -> main (forced update)


In [ ]:
# !python C:/Users/thuylt16/PycharmProjects/web_tracy/web_lor_tracy/web/data/crawl_db.py --input C:/Users/thuylt16/Downloads/failed_searches(2).csv --output results.csv --error-output errors.csv

In [1]:
# !pip install pandas


# Bước 1: Crawl YouGlish

In [2]:
import pandas as pd

df = pd.read_csv(r"C:\Users\thuyl\Downloads\failed_searches.csv")
words_to_search = df['word'].tolist()
print(words_to_search)
all_results = []
error_words = []
df_youglish = None
print(len(words_to_search))

['sinkhole', 'unify', 'conquer']
3


In [3]:
from web.data.crawl_db import search_youglish_full
import datetime
output = 'results.csv'
error_output = 'errors.csv'

consecutive_errors = 0
success_count = 0
error_count = 0

for i, word in enumerate(words_to_search):
    try:
        # Search YouGlish for the current word
        result = search_youglish_full(word)
        # Initialize or append to results dataframe
        if df_youglish is None:
            df_youglish = pd.DataFrame(result)
        else:
            df_youglish = pd.concat([df_youglish, pd.DataFrame(result)])
        consecutive_errors = 0
        success_count += 1
        print(f"Successfully processed word: {word}")
    except Exception as e:
        print(f"Error processing {word}")
        error_words.append(word)
        consecutive_errors += 1
        error_count += 1
        
        # If 3 consecutive errors, save and pause
        if consecutive_errors >= 3:
            print("3 consecutive errors encountered. Saving progress and pausing...")
            print(f"Total successful words: {success_count}")
            print(f"Total failed words: {error_count}")
            if df_youglish is not None:
                df_youglish = df_youglish[~df_youglish['caption'].str.contains('\(Applause\)', case=False, na=False)]
                df_youglish.to_csv(output, index=False)
            pd.DataFrame({'word': error_words}).to_csv(error_output, index=False)
            break
            
        continue
    
    # Save intermediate results every 100 words
    if i % 100 == 0 and i != 0:
        print(f"Processed {i} words")
        print(f"Successful words so far: {success_count}")
        print(f"Failed words so far: {error_count}")
        # Remove entries containing applause
        df_youglish = df_youglish[~df_youglish['caption'].str.contains('\(Applause\)', case=False, na=False)]
        # Save results and error words
        df_youglish.to_csv(output, index=False)
        pd.DataFrame({'word': error_words}).to_csv(error_output, index=False)

print(f"\nFinal Statistics:")
print(f"Total successful words: {success_count}")
print(f"Total failed words: {error_count}")
today = datetime.datetime.now().strftime("%Y%m%d")



sinkhole
Successfully processed word: sinkhole
unify
Successfully processed word: unify
conquer
Successfully processed word: conquer

Final Statistics:
Total successful words: 3
Total failed words: 0


In [4]:
df_youglish.to_csv(f'web/data/youglish_data{today}.csv', index=False)

In [5]:
error_words

[]

# bước 2: update data

In [9]:
import os
from datetime import datetime

today = datetime.now().strftime("%Y%m%d")

# Read both dataframes at once

df_old = pd.read_csv('web/data/youglish_data.csv')
df_new = pd.read_csv(f'youglish_data_{today}.csv')
print(df_old.shape)

# Concatenate and deduplicate in one step
df = pd.concat([df_old, df_new], ignore_index=True)
print(df.shape)
df = df.drop_duplicates(keep='last')
print(df.shape)




(55807, 6)
(57317, 6)
(55807, 6)


In [10]:
df.to_csv('web/data/youglish_data.csv', index=False)



In [11]:

#xóa file results.csv va youglish_data_20250116.csv
os.remove(f'youglish_data_{today}.csv')

In [19]:
from web.backend import find_id

find_id(['give something up', 'take something up', 'turn something', 'eat out'], df)


,video_id,video_title,start_second,end_second,caption,word
0,lTj6N1Im4FU,Poetry Slam Champion | Clint Smith | Talks at ...,256,261.0,"one could do was to give something up,",give something up
1,EGtGQhJYA8w,Saturday Series 2021- The Climate Emergency - ...,3418,3424.0,level that can really inspire people to take s...,take something up
2,IHE63fxpHCg,The biggest problem in AI? Machines have no co...,165,174.0,"I need to turn something, jiggle something -- ...",turn something


# b3 push data to redis

In [ ]:
import redis
import pandas as pd

REDIS_CONFIG = {
    'host': 'redis-11993.c228.us-central1-1.gce.redns.redis-cloud.com',
    'port': 11993,
    'decode_responses': True,
    'username': "default",
    'password': "ObIEQ5XPlF8xNDac7U7yiN6gtBUFVVUp",
}
redis_client = redis.Redis(**REDIS_CONFIG)


In [ ]:
def push_df_to_redis(df: pd.DataFrame, key: str, redis_client: redis.Redis):
    df.to_json(orient='records')
    redis_client.set(key, df.to_json(orient='records'))

df = pd.read_csv('web/data/youglish_data.csv')
push_df_to_redis(df, 'youglish_data', redis_client)

In [2]:

def get_df_from_redis(key: str, redis_client: redis.Redis) -> pd.DataFrame | None:
    """
    Retrieve DataFrame from Redis by key.
    
    Args:
        key: Redis key to retrieve data
        redis_client: Redis client instance
    
    Returns:
        DataFrame if data exists, None otherwise
    """
    
    json_data = redis_client.get(key)
    if json_data is not None:
        return pd.read_json(json_data, orient='records')
    return None

    
df = get_df_from_redis('youglish_data', redis_client)



C:\Users\thuylt16\AppData\Local\Temp\ipykernel_25252\3101844725.py:27: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_json(json_data, orient='records')


# Clean word

In [6]:
df = pd.read_csv(r"C:\Users\thuylt16\Downloads\B1_style.csv")
def clean_word(word: str) -> str:
    """Clean and format word string."""
    word = word.replace('<div>', '').replace('</div>', '')
    word = word.replace('(', '').replace(')', '').replace('"', '').strip()

    return word

df['word'] = df['word'].apply(clean_word)
df['word'].tolist()


['beat',
 'board game',
 'captain',
 'challenge',
 'challenge',
 'champion',
 'cheat',
 'classical music',
 'club',
 'coach',
 'competition',
 'concert',
 'defeat',
 'defeat',
 'entertaining',
 'folk music',
 'group',
 'gym',
 'have fun',
 'interest',
 'interest',
 'member',
 'opponent',
 'organize',
 'pleasure',
 'referee',
 'rhythm',
 'risk',
 'risk',
 'score',
 'score',
 'support',
 'support',
 'team',
 'train',
 'video game',
 'carry something on',
 'eat out',
 'give something up',
 'join in something',
 'sent off',
 'take something up',
 'turn something/someone down',
 'turn up somewhere',
 'for a long time',
 'for fun',
 'in the middle of',
 'in time for',
 'on CD/DVD/video',
 'on stage',
 'act',
 'athlete',
 'child',
 'collect',
 'entertain',
 'hero',
 'music',
 'play',
 'sail',
 'sing',
 'action',
 'inactive',
 'actor',
 'athletic',
 'athletics',
 'children',
 'childhood',
 'collection',
 'collector',
 'entertainment',
 'heroic',
 'heroine',
 'musical',
 'musician',
 'player',
